In [1]:
!pip install ray

In [2]:
!pip install jovian

In [3]:
#library imports
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
import ray.train as train
from ray.train import Trainer
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [4]:
import ray
ray.init(num_cpus=8)

{'metrics_export_port': 55625,
 'node_id': 'f203303f9abe63f5702c9edd583603cb40e4eeae11dad240ab745ad7',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2021-12-09_02-09-38_381968_3890/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2021-12-09_02-09-38_381968_3890/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2021-12-09_02-09-38_381968_3890',
 'webui_url': None}

In [5]:
from json import load
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
PATH ='/content/drive/MyDrive/BigData/'

In [42]:
! ls /root/ray_results/train_2021-12-09_02-18-44/run_001

In [8]:
with open(PATH + '/MMHS150K_GT.json', 'r') as fp:
  data = load(fp)

In [9]:
df = pd.DataFrame.from_dict(data, orient='index')

df = df[['tweet_text', 'labels', 'tweet_url']]

In [10]:
df['tweet_text'] = df['tweet_text'] + ' ' + df['tweet_text'] 

In [11]:
df.head()

,tweet_text,labels,tweet_url
1114679353714016256,@FriskDontMiss Nigga https://t.co/cAsaLWEpue @...,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...
1063020048816660480,My horses are retarded https://t.co/HYhqc6d5WN...,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...
1108927368075374593,“NIGGA ON MA MOMMA YOUNGBOY BE SPITTING REAL S...,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...
1114558534635618305,RT xxSuGVNGxx: I ran into this HOLY NIGGA TODA...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...
1035252480215592966,“EVERYbody calling you Nigger now!” https://t....,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...


In [12]:
_stopwords = set(stopwords.words("english") + ['\'', '’'])

def limpiar(t):
  without_links = ' '.join([word for word in t.split(' ') if word[:4] != 'http']) # quitar los links
  tokens = word_tokenize(without_links) # tokenizar (dividir en tokens)
  tokens = map(lambda token: token.encode('ascii', 'ignore').decode('ascii'), tokens)
  tokens_without_sw = [word.lower() for word in tokens if word.isalpha() and not word in _stopwords]
  return ' '.join(tokens_without_sw)

@ray.remote
def ray_limpiar(t):
  return limpiar(t)

In [13]:
df['tweet_text'] = df['tweet_text'].map(ray_limpiar.remote)

(ray_limpiar pid=4125) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4041) 
(ray_limpiar pid=4173) 
(ray_limpiar pid=4183) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4124) 


(ray_limpiar pid=4042) 


(ray_limpiar pid=4124) 
(ray_limpiar pid=4042) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4124) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4239) 
(ray_limpiar pid=4238) 
(ray_limpiar pid=4125) 
(ray_limpiar pid=4124) 
(ray_limpiar pid=4183) 
(ray_limpiar pid=4042) 


(ray_limpiar pid=4042) 
(ray_limpiar pid=4041) 


(ray_limpiar pid=4041) 
(ray_limpiar pid=4238) 
(ray_limpiar pid=4042) 
(ray_limpiar pid=4239) 


...

(ray_limpiar pid=4238) 
(ray_limpiar pid=4173) 
(ray_limpiar pid=4041) 


(ray_limpiar pid=4183) 


(ray_limpiar pid=4124) 
(ray_limpiar pid=4183) 
(ray_limpiar pid=4042) 


(ray_limpiar pid=4183) 


(ray_limpiar pid=4125) 


In [14]:
df['tweet_text'] = df['tweet_text'].map(ray.get)

In [15]:
df.head()

,tweet_text,labels,tweet_url
1114679353714016256,friskdontmiss nigga friskdontmiss nigga,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...
1063020048816660480,my horses retarded my horses retarded,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...
1108927368075374593,nigga on ma momma youngboy be spitting real sh...,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...
1114558534635618305,rt xxsugvngxx i ran holy nigga today rt xxsugv...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...
1035252480215592966,everybody calling nigger everybody calling nigger,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...


In [16]:
#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [17]:
counts = Counter()
for index, row in df.iterrows():
    counts.update(tokenize(row['tweet_text']))

In [18]:
#creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [19]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [20]:
def encode_label(labels):
    zeroes_count = 0
    for i in labels:
      if i == 0:
        zeroes_count += 1
    if zeroes_count > 1:
      return 0
    
    return 1

In [21]:
df['encoded'] = df['tweet_text'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df['targer_label'] = df['labels'].apply(lambda x: encode_label(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [22]:
df['encoded']

1114679353714016256    [[2, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1063020048816660480    [[4, 5, 6, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1108927368075374593    [[3, 7, 8, 9, 10, 11, 12, 13, 14, 3, 3, 7, 8, ...
1114558534635618305    [[15, 16, 17, 18, 19, 3, 20, 15, 16, 17, 18, 1...
1035252480215592966    [[21, 22, 23, 21, 22, 23, 0, 0, 0, 0, 0, 0, 0,...
                                             ...                        
1114170734472048640    [[83172, 17, 345, 470, 223, 67, 18676, 222, 83...
1110368198786846720    [[3792, 3, 849, 15901, 3792, 3, 849, 15901, 0,...
1106941858540851200    [[4, 3, 199, 2549, 4, 3, 199, 2549, 0, 0, 0, 0...
1105268309233188865    [[470, 3, 387, 250, 114, 8575, 83173, 9603, 25...
1114653514364530691    [[75, 3, 4896, 10143, 114, 207, 182, 75310, 24...
Name: encoded, Length: 149823, dtype: object

In [32]:
df['length'] = df.tweet_text.str.len()

In [33]:
df = df[df.length > 0]

In [34]:
X = list(df['encoded'])
y = list(df['targer_label'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [35]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [36]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [37]:
def train_model():
    model = LSTM_variable_input(vocab_size, 50, 50)
    epochs=30
    lr=0.1
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()
            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        if i % 5 == 1:
            print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [38]:
batch_size = 5000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [39]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

In [29]:
df['length'] = df.tweet_text.str.len()


In [43]:
df.head()

,tweet_text,labels,tweet_url,encoded,targer_label,length
1114679353714016256,friskdontmiss nigga friskdontmiss nigga,"[4, 1, 3]",https://twitter.com/user/status/11146793537140...,"[[2, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,39
1063020048816660480,my horses retarded my horses retarded,"[5, 5, 5]",https://twitter.com/user/status/10630200488166...,"[[4, 5, 6, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1,37
1108927368075374593,nigga on ma momma youngboy be spitting real sh...,"[0, 0, 0]",https://twitter.com/user/status/11089273680753...,"[[3, 7, 8, 9, 10, 11, 12, 13, 14, 3, 3, 7, 8, ...",0,109
1114558534635618305,rt xxsugvngxx i ran holy nigga today rt xxsugv...,"[1, 0, 0]",https://twitter.com/user/status/11145585346356...,"[[15, 16, 17, 18, 19, 3, 20, 15, 16, 17, 18, 1...",0,73
1035252480215592966,everybody calling nigger everybody calling nigger,"[1, 0, 1]",https://twitter.com/user/status/10352524802155...,"[[21, 22, 23, 21, 22, 23, 0, 0, 0, 0, 0, 0, 0,...",1,49


In [30]:
df = df[df.length > 0]

In [ ]:
for i in df['tweet_text']:
  if()

In [40]:
trainer = Trainer(backend="torch", num_workers=2)
trainer.start()
results = trainer.run(train_model)
trainer.shutdown()

2021-12-09 02:18:44,258	INFO trainer.py:172 -- Trainer logs will be logged in: /root/ray_results/train_2021-12-09_02-18-44
(BaseWorkerMixin pid=4124) 2021-12-09 02:18:48,426	INFO torch.py:67 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=4239) 2021-12-09 02:18:48,410	INFO torch.py:67 -- Setting up process group for: env:// [rank=1, world_size=2]
2021-12-09 02:18:49,426	INFO trainer.py:178 -- Run results will be logged in: /root/ray_results/train_2021-12-09_02-18-44/run_001


(BaseWorkerMixin pid=4239) train loss 0.481, val loss 0.519, val accuracy 0.760, and val rmse 0.490
(BaseWorkerMixin pid=4124) train loss 0.497, val loss 0.539, val accuracy 0.753, and val rmse 0.497
(BaseWorkerMixin pid=4124) train loss 0.342, val loss 0.632, val accuracy 0.731, and val rmse 0.519
(BaseWorkerMixin pid=4239) train loss 0.328, val loss 0.646, val accuracy 0.737, and val rmse 0.513
(BaseWorkerMixin pid=4124) train loss 0.301, val loss 0.694, val accuracy 0.732, and val rmse 0.517
(BaseWorkerMixin pid=4239) train loss 0.291, val loss 0.702, val accuracy 0.730, and val rmse 0.520
(BaseWorkerMixin pid=4124) train loss 0.292, val loss 0.731, val accuracy 0.731, and val rmse 0.518
(BaseWorkerMixin pid=4239) train loss 0.277, val loss 0.730, val accuracy 0.736, and val rmse 0.513
(BaseWorkerMixin pid=4124) train loss 0.285, val loss 0.735, val accuracy 0.731, and val rmse 0.519
(BaseWorkerMixin pid=4239) train loss 0.273, val loss 0.737, val accuracy 0.731, and val rmse 0.519
